In [1]:
import nltk

In [2]:
a = nltk.corpus.BracketParseCorpusReader("SST", "(train|dev|test)\.txt")

In [4]:
text = {}
labels = {}
keys = ['train', 'dev', 'test']
for k in keys :
    text[k] = [x.leaves() for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    labels[k] = [int(x.label()) for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    print(len(text[k]))

8544
1101
2210


In [4]:
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
import re

In [5]:
def tokenize(text) :
    text = " ".join(text)
    text = text.replace("-LRB-", '')
    text = text.replace("-RRB-", " ")
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    tokens = " ".join([t.text.lower() for t in nlp(text)])
    return tokens

for k in keys :
    text[k] = [tokenize(t) for t in text[k]]

In [6]:
import vectorizer
vec = vectorizer.Vectorizer(min_df=None)

In [7]:
vec.fit([x for k in text for x in text[k]])

In [8]:
vec.seq_text = {}
vec.label = {}
for k in keys :
    vec.seq_text[k] = vec.get_seq_for_docs(text[k])
    vec.label[k] = [1 if x >= 3 else 0 for x in labels[k]]

In [1]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [10]:
vec.word_dim = vectors.dim

In [11]:
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))

In [12]:
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [13]:
import pickle
pickle.dump(vec, open('SST/sst.p', 'wb'))